In [2]:
'''
This code outputs charts showing trading activities for different ASX electricity contracts.

Users can choose to plot any or all of the following data types:
- Daily average price
- Cumulative average price
- Daily MWh traded
- Cumulative MWh traded
for the ASX electricity contracts for the following states
- New South Wales
- Victoria
- Queensland
- South Australia
for the following contract types
- Base
- Peak
- Cap ($300/MWh)
for the following expiry time
- Q1 (last day of Q1 of the year)
- Q2 (last day of Q2 of the year)
- Q3 (last day of Q3 of the year)
- Q4 (last day of Q4 of the year)
for the following years (this can be extended with a bigger input dataset)
- 2022, 2023, 2024, 2025, 2026
against time which can either be
- Actual calendar days or
- Number of days from contract expiry
and choose to compare by either the expiry quarter, expiry year, state or contract type.
'''

'\nThis code outputs charts showing trading activities for different ASX electricity contracts.\n\nUsers can choose to plot any or all of the following data types:\n- Daily average price\n- Cumulative average price\n- Daily MWh traded\n- Cumulative MWh traded\nfor the ASX electricity contracts for the following states\n- New South Wales\n- Victoria\n- Queensland\n- South Australia\nfor the following contract types\n- Base\n- Peak\n- Cap ($300/MWh)\nfor the following expiry time\n- Q1 (last day of Q1 of the year)\n- Q2 (last day of Q2 of the year)\n- Q3 (last day of Q3 of the year)\n- Q4 (last day of Q4 of the year)\nfor the following years (this can be extended with a bigger input dataset)\n- 2022, 2023, 2024, 2025, 2026\nagainst time which can either be\n- Actual calendar days or\n- Number of days from contract expiry\nand choose to compare by either the expiry quarter, expiry year, state or contract type.\n'

In [4]:
# Import libraries
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [6]:
# Update the file path for the trade summary data
file_path = r'C:\Users\wwang2\OneDrive - ACCC\Desktop\Grad\R4_AER_EPR\Analysing price and volume trends in ASX energy contracts\summary\trades_summary.csv'

In [9]:
summary_df = pd.read_csv(file_path)
summary_df['Date'] = pd.to_datetime(summary_df['Date'])

In [10]:
def plot_comparison_by_quarter(df, data_types, states, contract_types, quarters, years, time_criteria):
    for state in states:
        for contract_type in contract_types:
            for year in years:
                fig, ax1 = plt.subplots(figsize=(18, 16), dpi=150)
                
                # Determine the type of plot based on data_types
                contains_price = any(dt in ['Daily average price', 'Cumulative average price'] for dt in data_types)
                contains_mwh = any(dt in ['Daily MWh traded', 'Cumulative MWh traded'] for dt in data_types)
                
                if contains_price and contains_mwh:
                    ax2 = ax1.twinx()
                else:
                    ax2 = None
                
                for quarter in quarters:
                    for dt in data_types:
                        days_from_expiry = f'Days from expiry for {contract_type} {state} {quarter} {year}'
                        column_name = f'{dt} for {contract_type} {state} {quarter} {year}'
                        if column_name in df.columns:
                            if dt in ['Daily average price', 'Cumulative average price']:
                                if time_criteria == 'Calendar Dates':
                                    ax1.plot(df['Date'], df[column_name], label=f'{quarter} - {dt}')
                                else:
                                    ax1.plot(-df[days_from_expiry], df[column_name], label=f'{quarter} - {dt}')
                            elif dt in ['Daily MWh traded', 'Cumulative MWh traded']:
                                if ax2:
                                    if time_criteria == 'Calendar Dates':
                                        ax2.plot(df['Date'], df[column_name], label=f'{quarter} - {dt}', linestyle='--')
                                    else:
                                        ax2.plot(-df[days_from_expiry], df[column_name], label=f'{quarter} - {dt}', linestyle='--')
                                else:
                                    if time_criteria == 'Calendar Dates':
                                        ax1.plot(df['Date'], df[column_name], label=f'{quarter} - {dt}')
                                    else:
                                        ax1.plot(-df[days_from_expiry], df[column_name], label=f'{quarter} - {dt}')
                
                ax1.set_title(f'{state} {contract_type} {year} trades summary')
                ax1.set_xlabel('Date' if time_criteria == 'Calendar Dates' else 'Days from expiry')
                
                if contains_price:
                    ax1.set_ylabel('Average price ($/MWh)')
                if contains_mwh and ax2:
                    ax2.set_ylabel('MWh traded')
                elif contains_mwh:
                    ax1.set_ylabel('MWh traded')
                
                lines, labels = ax1.get_legend_handles_labels()
                if ax2:
                    lines2, labels2 = ax2.get_legend_handles_labels()
                    lines += lines2
                    labels += labels2
                ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
                
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.savefig(f'summary/{state} {contract_type} {year} trades summary - {quarter}.png', bbox_inches='tight')
                plt.show()
                plt.close()

In [11]:
def plot_comparison_by_year(df, data_types, states, contract_types, quarters, years, time_criteria):
    for state in states:
        for contract_type in contract_types:
            for quarter in quarters:
                fig, ax1 = plt.subplots(figsize=(18, 16), dpi=150)
                
                # Determine the type of plot based on data_types
                contains_price = any(dt in ['Daily average price', 'Cumulative average price'] for dt in data_types)
                contains_mwh = any(dt in ['Daily MWh traded', 'Cumulative MWh traded'] for dt in data_types)
                
                if contains_price and contains_mwh:
                    ax2 = ax1.twinx()
                else:
                    ax2 = None
                
                for year in years:
                    for dt in data_types:
                        days_from_expiry = f'Days from expiry for {contract_type} {state} {quarter} {year}'
                        column_name = f'{dt} for {contract_type} {state} {quarter} {year}'
                        if column_name in df.columns:
                            if dt in ['Daily average price', 'Cumulative average price']:
                                if time_criteria == 'Calendar Dates':
                                    ax1.plot(df['Date'], df[column_name], label=f'{year} - {dt}')
                                else:
                                    ax1.plot(-df[days_from_expiry], df[column_name], label=f'{year} - {dt}')
                            elif dt in ['Daily MWh traded', 'Cumulative MWh traded']:
                                if ax2:
                                    if time_criteria == 'Calendar Dates':
                                        ax2.plot(df['Date'], df[column_name], label=f'{year} - {dt}', linestyle='--')
                                    else:
                                        ax2.plot(-df[days_from_expiry], df[column_name], label=f'{year} - {dt}', linestyle='--')
                                else:
                                    if time_criteria == 'Calendar Dates':
                                        ax1.plot(df['Date'], df[column_name], label=f'{year} - {dt}')
                                    else:
                                        ax1.plot(-df[days_from_expiry], df[column_name], label=f'{year} - {dt}')
                
                ax1.set_title(f'{state} {contract_type} {quarter} trades summary')
                ax1.set_xlabel('Date' if time_criteria == 'Calendar Dates' else 'Days from expiry')
                
                if contains_price:
                    ax1.set_ylabel('Average price ($/MWh)')
                if contains_mwh and ax2:
                    ax2.set_ylabel('MWh traded')
                elif contains_mwh:
                    ax1.set_ylabel('MWh traded')
                
                lines, labels = ax1.get_legend_handles_labels()
                if ax2:
                    lines2, labels2 = ax2.get_legend_handles_labels()
                    lines += lines2
                    labels += labels2
                ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
                
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.savefig(f'summary/{state} {contract_type} {quarter} trades summary - {year}.png', bbox_inches='tight')
                plt.show()
                plt.close()

In [13]:
def plot_comparison_by_state(df, data_types, states, contract_types, quarters, years, time_criteria):
    for contract_type in contract_types:
        for year in years:
            for quarter in quarters:
                fig, ax1 = plt.subplots(figsize=(18, 16), dpi=150)
                
                # Determine the type of plot based on data_types
                contains_price = any(dt in ['Daily average price', 'Cumulative average price'] for dt in data_types)
                contains_mwh = any(dt in ['Daily MWh traded', 'Cumulative MWh traded'] for dt in data_types)
                
                if contains_price and contains_mwh:
                    ax2 = ax1.twinx()
                else:
                    ax2 = None
                
                for state in states:
                    for dt in data_types:
                        days_from_expiry = f'Days from expiry for {contract_type} {state} {quarter} {year}'
                        column_name = f'{dt} for {contract_type} {state} {quarter} {year}'
                        if column_name in df.columns:
                            if dt in ['Daily average price', 'Cumulative average price']:
                                if time_criteria == 'Calendar Dates':
                                    ax1.plot(df['Date'], df[column_name], label=f'{state} - {dt}')
                                else:
                                    ax1.plot(-df[days_from_expiry], df[column_name], label=f'{state} - {dt}')
                            elif dt in ['Daily MWh traded', 'Cumulative MWh traded']:
                                if ax2:
                                    if time_criteria == 'Calendar Dates':
                                        ax2.plot(df['Date'], df[column_name], label=f'{state} - {dt}', linestyle='--')
                                    else:
                                        ax2.plot(-df[days_from_expiry], df[column_name], label=f'{state} - {dt}', linestyle='--')
                                else:
                                    if time_criteria == 'Calendar Dates':
                                        ax1.plot(df['Date'], df[column_name], label=f'{state} - {dt}')
                                    else:
                                        ax1.plot(-df[days_from_expiry], df[column_name], label=f'{state} - {dt}')
                
                ax1.set_title(f'{contract_type} {year} {quarter} trades summary')
                ax1.set_xlabel('Date' if time_criteria == 'Calendar Dates' else 'Days from expiry')
                
                if contains_price:
                    ax1.set_ylabel('Average price ($/MWh)')
                if contains_mwh and ax2:
                    ax2.set_ylabel('MWh traded')
                elif contains_mwh:
                    ax1.set_ylabel('MWh traded')
                
                lines, labels = ax1.get_legend_handles_labels()
                if ax2:
                    lines2, labels2 = ax2.get_legend_handles_labels()
                    lines += lines2
                    labels += labels2
                ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
                
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.savefig(f'summary/{contract_type} {year} {quarter} trades summary - {state}.png', bbox_inches='tight')
                plt.show()
                plt.close()

In [14]:
def plot_comparison_by_contract_type(df, data_types, states, contract_types, quarters, years, time_criteria):
    for state in states:
        for year in years:
            for quarter in quarters:
                fig, ax1 = plt.subplots(figsize=(18, 16), dpi=150)
                
                # Determine the type of plot based on data_types
                contains_price = any(dt in ['Daily average price', 'Cumulative average price'] for dt in data_types)
                contains_mwh = any(dt in ['Daily MWh traded', 'Cumulative MWh traded'] for dt in data_types)
                
                if contains_price and contains_mwh:
                    ax2 = ax1.twinx()
                else:
                    ax2 = None
                
                for contract_type in contract_types:
                    for dt in data_types:
                        days_from_expiry = f'Days from expiry for {contract_type} {state} {quarter} {year}'
                        column_name = f'{dt} for {contract_type} {state} {quarter} {year}'
                        if column_name in df.columns:
                            if dt in ['Daily average price', 'Cumulative average price']:
                                if time_criteria == 'Calendar Dates':
                                    ax1.plot(df['Date'], df[column_name], label=f'{contract_type} - {dt}')
                                else:
                                    ax1.plot(-df[days_from_expiry], df[column_name], label=f'{contract_type} - {dt}')
                            elif dt in ['Daily MWh traded', 'Cumulative MWh traded']:
                                if ax2:
                                    if time_criteria == 'Calendar Dates':
                                        ax2.plot(df['Date'], df[column_name], label=f'{contract_type} - {dt}', linestyle='--')
                                    else:
                                        ax2.plot(-df[days_from_expiry], df[column_name], label=f'{contract_type} - {dt}', linestyle='--')
                                else:
                                    if time_criteria == 'Calendar Dates':
                                        ax1.plot(df['Date'], df[column_name], label=f'{contract_type} - {dt}')
                                    else:
                                        ax1.plot(-df[days_from_expiry], df[column_name], label=f'{contract_type} - {dt}')
                
                ax1.set_title(f'{state} {year} {quarter} trades summary')
                ax1.set_xlabel('Date' if time_criteria == 'Calendar Dates' else 'Days from expiry')
                
                if contains_price:
                    ax1.set_ylabel('Average price ($/MWh)')
                if contains_mwh and ax2:
                    ax2.set_ylabel('MWh traded')
                elif contains_mwh:
                    ax1.set_ylabel('MWh traded')
                
                lines, labels = ax1.get_legend_handles_labels()
                if ax2:
                    lines2, labels2 = ax2.get_legend_handles_labels()
                    lines += lines2
                    labels += labels2
                ax1.legend(lines, labels, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=2)
                
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                plt.savefig(f'summary/{state} {year} {quarter} trades summary - {contract_type}.png', bbox_inches='tight')
                plt.show()
                plt.close()

In [15]:
# Adjusted function to compare data by user-selected criteria and display multiple data types in the same chart
def plot_comparison(df, data_types, states, contract_types, quarters, years, compare_by, time_criteria):
    if compare_by == 'Quarter':
        plot_comparison_by_quarter(df, data_types, states, contract_types, quarters, years, time_criteria)
    elif compare_by == 'Year':
        plot_comparison_by_year(df, data_types, states, contract_types, quarters, years, time_criteria)
    elif compare_by == 'State':
        plot_comparison_by_state(df, data_types, states, contract_types, quarters, years, time_criteria)
    elif compare_by == 'Contract Type':
        plot_comparison_by_contract_type(df, data_types, states, contract_types, quarters, years, time_criteria)

# Interactive widgets
data_type_widget = widgets.SelectMultiple(options=['Daily average price', 'Cumulative average price', 'Daily MWh traded', 'Cumulative MWh traded'], description='Data types')
state_widget = widgets.SelectMultiple(options=['NSW', 'VIC', 'QLD', 'SA'], description='States')
contract_type_widget = widgets.SelectMultiple(options=['Base', 'Peak', 'Cap'], description='Contract types')
quarter_widget = widgets.SelectMultiple(options=['Q1', 'Q2', 'Q3', 'Q4'], description='Quarters')
year_widget = widgets.SelectMultiple(options=['2022', '2023', '2024', '2025', '2026'], description='Years')
compare_by_widget = widgets.RadioButtons(options=['Quarter', 'Year', 'State', 'Contract Type'], description='Compare by')
time_widget = widgets.RadioButtons(options=['Calendar Dates', 'Days from Expiry'], description='Time Criteria')

# Display widgets
display(data_type_widget, state_widget, contract_type_widget, quarter_widget, year_widget, compare_by_widget, time_widget)

# Button to trigger the plot
button = widgets.Button(description="Plot Comparison")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        plot_comparison(summary_df, data_type_widget.value, state_widget.value, contract_type_widget.value, quarter_widget.value, year_widget.value, compare_by_widget.value, time_widget.value)

button.on_click(on_button_clicked)
display(button, output)

SelectMultiple(description='Data types', options=('Daily average price', 'Cumulative average price', 'Daily MW…

SelectMultiple(description='States', options=('NSW', 'VIC', 'QLD', 'SA'), value=())

SelectMultiple(description='Contract types', options=('Base', 'Peak', 'Cap'), value=())

SelectMultiple(description='Quarters', options=('Q1', 'Q2', 'Q3', 'Q4'), value=())

SelectMultiple(description='Years', options=('2022', '2023', '2024', '2025', '2026'), value=())

RadioButtons(description='Compare by', options=('Quarter', 'Year', 'State', 'Contract Type'), value='Quarter')

RadioButtons(description='Time Criteria', options=('Calendar Dates', 'Days from Expiry'), value='Calendar Date…

Button(description='Plot Comparison', style=ButtonStyle())

Output()